# Calculate Fishing Days of Chinese Fleet and North Korean Fleet
This notebook calculates the total fishing days of the Chinese fleet and North Korean fleet based on the satellite detections as well as the South Korean Coast Guard's count of vessels travelling from South to North Korean waters or vice versa. All datasets used in the paper "Illuminating Dark Fishing Fleets in North Korea" published in Science Advances on July 22, 2020 are publicly available under `global-fishing-watch.paper_dark_fishing_fleets_in_north_korea` BigQuery bucket.

Last updated: 23 June 2020

Developed by Global Fishing Watch Inc.

In [1]:
#
# Import libraries
#
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import calendar
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

import gdal
import fiona
from shapely.geometry import shape, Point

In [2]:
#
# Detection boundaries
#
_eez_prk_wo_mz = '../1-boundary/eez_prk_wo_mz/eez_prk_wo_mz'
_eez_prk_mz = '../1-boundary/eez_prk_mz/eez_prk_mz'
_study_area = '../1-boundary/study_area/study_area'
_eez_rus_near_prk_wo_12nm = '../1-boundary/eez_rus_near_prk_wo_12nm/eez_rus_near_prk_wo_12nm'

## Estimated total number of fishing days derived from satellite detections

In [3]:
q = """
WITH 
  #
  # Import all satellite detection data and put them in half-month bins
  #
  data AS (
    SELECT 
      *,
      CASE  
        WHEN day_of_year BETWEEN 1 AND 15 THEN "01.1"
        WHEN day_of_year BETWEEN 16 AND 31 THEN "01.2"
        WHEN day_of_year BETWEEN 32 AND 46 THEN "02.1"
        WHEN day_of_year BETWEEN 47 AND 59 THEN "02.2"
        WHEN day_of_year BETWEEN 60 AND 74 THEN "03.1"
        WHEN day_of_year BETWEEN 75 AND 90 THEN "03.2"
        WHEN day_of_year BETWEEN 91 AND 105 THEN "04.1"
        WHEN day_of_year BETWEEN 106 AND 120 THEN "04.2"
        WHEN day_of_year BETWEEN 121 AND 135 THEN "05.1"
        WHEN day_of_year BETWEEN 136 AND 151 THEN "05.2"
        WHEN day_of_year BETWEEN 152 AND 166 THEN "06.1"
        WHEN day_of_year BETWEEN 167 AND 181 THEN "06.2"
        WHEN day_of_year BETWEEN 182 AND 196 THEN "07.1"
        WHEN day_of_year BETWEEN 197 AND 212 THEN "07.2"
        WHEN day_of_year BETWEEN 213 AND 227 THEN "08.1"
        WHEN day_of_year BETWEEN 228 AND 243 THEN "08.2"
        WHEN day_of_year BETWEEN 244 AND 258 THEN "09.1"
        WHEN day_of_year BETWEEN 259 AND 273 THEN "09.2"
        WHEN day_of_year BETWEEN 274 AND 288 THEN "10.1"
        WHEN day_of_year BETWEEN 289 AND 304 THEN "10.2"
        WHEN day_of_year BETWEEN 305 AND 319 THEN "11.1"
        WHEN day_of_year BETWEEN 320 AND 334 THEN "11.2"
        WHEN day_of_year BETWEEN 335 AND 349 THEN "12.1"
        WHEN day_of_year BETWEEN 350 AND 365 THEN "12.2"
        ELSE NULL
      END AS bin
    FROM `paper_dark_fishing_fleets_in_north_korea.daily_number_of_detections`
  ),
  
  #
  # Import data for the leap year 2016
  #
  data_leapyear AS (
    SELECT 
      *,
      CASE  
        WHEN day_of_year BETWEEN 1 AND 15 THEN "01.1"
        WHEN day_of_year BETWEEN 16 AND 31 THEN "01.2"
        WHEN day_of_year BETWEEN 32 AND 46 THEN "02.1"
        WHEN day_of_year BETWEEN 47 AND 60 THEN "02.2"
        WHEN day_of_year BETWEEN 61 AND 75 THEN "03.1"
        WHEN day_of_year BETWEEN 76 AND 91 THEN "03.2"
        WHEN day_of_year BETWEEN 92 AND 106 THEN "04.1"
        WHEN day_of_year BETWEEN 107 AND 121 THEN "04.2"
        WHEN day_of_year BETWEEN 122 AND 136 THEN "05.1"
        WHEN day_of_year BETWEEN 137 AND 152 THEN "05.2"
        WHEN day_of_year BETWEEN 153 AND 167 THEN "06.1"
        WHEN day_of_year BETWEEN 168 AND 182 THEN "06.2"
        WHEN day_of_year BETWEEN 183 AND 197 THEN "07.1"
        WHEN day_of_year BETWEEN 198 AND 213 THEN "07.2"
        WHEN day_of_year BETWEEN 214 AND 228 THEN "08.1"
        WHEN day_of_year BETWEEN 228 AND 244 THEN "08.2"
        WHEN day_of_year BETWEEN 245 AND 259 THEN "09.1"
        WHEN day_of_year BETWEEN 260 AND 274 THEN "09.2"
        WHEN day_of_year BETWEEN 275 AND 289 THEN "10.1"
        WHEN day_of_year BETWEEN 290 AND 305 THEN "10.2"
        WHEN day_of_year BETWEEN 306 AND 320 THEN "11.1"
        WHEN day_of_year BETWEEN 321 AND 335 THEN "11.2"
        WHEN day_of_year BETWEEN 336 AND 350 THEN "12.1"
        WHEN day_of_year BETWEEN 351 AND 366 THEN "12.2"
        ELSE NULL
      END AS bin
    FROM `paper_dark_fishing_fleets_in_north_korea.daily_number_of_detections`
  ),
  
  #
  # Number of days within each half-month window
  #
  num_days AS (
    SELECT "01.1" AS bin, 15 AS days UNION ALL
    SELECT "01.2", 16 UNION ALL
    SELECT "02.1", 15 UNION ALL
    SELECT "02.2", 13 UNION ALL
    SELECT "03.1", 15 UNION ALL
    SELECT "03.2", 16 UNION ALL
    SELECT "04.1", 15 UNION ALL
    SELECT "04.2", 15 UNION ALL
    SELECT "05.1", 15 UNION ALL
    SELECT "05.2", 16 UNION ALL
    SELECT "06.1", 15 UNION ALL
    SELECT "06.2", 15 UNION ALL
    SELECT "07.1", 15 UNION ALL
    SELECT "07.2", 16 UNION ALL
    SELECT "08.1", 15 UNION ALL
    SELECT "08.2", 16 UNION ALL
    SELECT "09.1", 15 UNION ALL
    SELECT "09.2", 15 UNION ALL
    SELECT "10.1", 15 UNION ALL
    SELECT "10.2", 16 UNION ALL
    SELECT "11.1", 15 UNION ALL
    SELECT "11.2", 15 UNION ALL
    SELECT "12.1", 15 UNION ALL
    SELECT "12.2", 16
  ),
  
  #
  # Number of days in each half-month window for the leap year 2016
  #
  num_days_leapyear AS (
    SELECT "01.1" AS bin, 15 AS days UNION ALL
    SELECT "01.2", 16 UNION ALL
    SELECT "02.1", 15 UNION ALL
    SELECT "02.2", 14 UNION ALL
    SELECT "03.1", 15 UNION ALL
    SELECT "03.2", 16 UNION ALL
    SELECT "04.1", 15 UNION ALL
    SELECT "04.2", 15 UNION ALL
    SELECT "05.1", 15 UNION ALL
    SELECT "05.2", 16 UNION ALL
    SELECT "06.1", 15 UNION ALL
    SELECT "06.2", 15 UNION ALL
    SELECT "07.1", 15 UNION ALL
    SELECT "07.2", 16 UNION ALL
    SELECT "08.1", 15 UNION ALL
    SELECT "08.2", 16 UNION ALL
    SELECT "09.1", 15 UNION ALL
    SELECT "09.2", 15 UNION ALL
    SELECT "10.1", 15 UNION ALL
    SELECT "10.2", 16 UNION ALL
    SELECT "11.1", 15 UNION ALL
    SELECT "11.2", 15 UNION ALL
    SELECT "12.1", 15 UNION ALL
    SELECT "12.2", 16
  ),
  
  #
  # Maximum daily number in each half-month period in 2017
  # for Chinese pair trawlers (detected by radar and optical satellite sensors)
  #
  chn_trawlers2017 AS (
    SELECT 
      bin,
      MAX(detection) AS half_month_max
    FROM data
    WHERE year = 2017
      AND flag = "chn"
      AND sensor IN ("sentinel1", "palsar2", "radarsat2", "planet")
    GROUP BY bin
  ),
  
  #
  # Estimated total fishing days in 2017
  # for Chinese pair trawlers (detected by radar and optical satellite sensors)
  #
  chn_trawlers2017_fdays AS (
    SELECT bin, half_month_max * days AS total
    FROM chn_trawlers2017
    LEFT JOIN num_days
    USING (bin)
  ),
  
  #
  # Maximum daily number in each half-month period in 2018
  # for Chinese pair trawlers (detected by radar and optical satellite sensors)
  #
  chn_trawlers2018 AS (
    SELECT 
      bin,
      MAX(detection) AS half_month_max
    FROM data
    WHERE year = 2018
      AND flag = "chn"
      AND sensor IN ("sentinel1", "palsar2", "radarsat2", "planet")
    GROUP BY bin
  ),

  #
  # Estimated total fishing days in 2018
  # for Chinese pair trawlers (detected by radar and optical satellite sensors)
  #
  chn_trawlers2018_fdays AS (
    SELECT bin, half_month_max * days AS total
    FROM chn_trawlers2018
    LEFT JOIN num_days
    USING (bin)
  ),
  
  #
  # Maximum daily number in each half-month period in 2017
  # for Chinese lighting vessels (detected by light time optical imagery)
  #
  chn_lighting2017 AS (
    SELECT 
      bin,
      MAX(detection) AS half_month_max
    FROM data
    WHERE year = 2017
      AND flag = "chn"
      AND sensor IN ("viirs")
    GROUP BY bin
  ),

  #
  # Estimated total fishing days in 2017
  # for Chinese lighting vessels (detected by light time optical imagery)
  #
  chn_lighting2017_fdays AS (
    SELECT bin, half_month_max * days AS total
    FROM chn_lighting2017
    LEFT JOIN num_days
    USING (bin)
  ),
  
  #
  # Maximum daily number in each half-month period in 2018
  # for Chinese lighting vessels (detected by light time optical imagery)
  #
  chn_lighting2018 AS (
    SELECT 
      bin,
      MAX(detection) AS half_month_max
    FROM data
    WHERE year = 2018
      AND flag = "chn"
      AND sensor IN ("viirs")
    GROUP BY bin
  ),

  #
  # Estimated total fishing days in 2018
  # for Chinese lighting vessels (detected by light time optical imagery)
  #
  chn_lighting2018_fdays AS (
    SELECT bin, half_month_max * days AS total
    FROM chn_lighting2018
    LEFT JOIN num_days
    USING (bin)
  ),
  
  #
  # Estimate total fishing days in 2017 for all Chinese vessels
  #
  chn_total2017_fdays AS (
    SELECT 
      'chn' AS flag, 2017 AS year, cat, 
      SUM(total) AS total, FALSE as sanction
    FROM (
      SELECT
        "trawler" AS cat,
        total
      FROM chn_trawlers2017_fdays 
      UNION ALL
      SELECT
        "lighting" AS cat,
        total
      FROM chn_lighting2017_fdays )
    GROUP BY cat
  ),
  
  #
  # Estimate total fishing days in 2017 for all Chinese vessels
  # after the sanctions entered into effect
  #
  chn_total2017_fdays_sanction AS (
    SELECT 
      'chn' AS flag, 2017 AS year, cat, 
      SUM(total) AS total, TRUE AS sanction
    FROM (
      SELECT
        "trawler" AS cat,
        CASE 
          WHEN bin = "09.1" THEN total * 11 / 15
          WHEN bin IN ("09.2", "10.1", "10.2", "11.1", "11.2", "12.1", "12.2") THEN total
          ELSE 0
        END AS total
      FROM chn_trawlers2017_fdays 
      UNION ALL
      SELECT
        "lighting" AS cat,
        CASE 
          WHEN bin = "09.1" THEN total * 11 / 15
          WHEN bin IN ("09.2", "10.1", "10.2", "11.1", "11.2", "12.1", "12.2") THEN total
          ELSE 0
        END AS total
      FROM chn_lighting2017_fdays )
    GROUP BY cat
  ),
  
  #
  # Estimate total fishing days in 2018 for all Chinese vessels
  #
  chn_total2018_fdays AS (
    SELECT 
      'chn' AS flag, 2018 AS year, cat, 
      SUM(total) AS total, TRUE as sanction
    FROM (
      SELECT
        "trawler" AS cat,
        total
      FROM chn_trawlers2018_fdays 
      UNION ALL
      SELECT
        "lighting" AS cat,
        total
      FROM chn_lighting2018_fdays )
    GROUP BY cat
  ),
  
  
  #
  # Maximum daily number in each half-month period in 2015
  # for North Korean lighting vessels (detected by light time optical imagery)
  #
  prk_lighting2015 AS (
    SELECT 
      bin,
      MAX(detection) AS half_month_max
    FROM data
    WHERE year = 2015
      AND flag = "prk"
      AND sensor IN ("viirs")
    GROUP BY bin
  ),
  
  #
  # Estimated total fishing days in 2015
  # for North Korean lighting vessels (detected by light time optical imagery)
  #
  prk_lighting2015_fdays AS (
    SELECT
      'prk' AS flag, 2015 AS year, 'lighting' AS cat,
      SUM(total) AS total, CAST(NULL AS BOOLEAN) AS sanction
    FROM (
      SELECT 
        bin, half_month_max * days AS total
      FROM prk_lighting2015
      LEFT JOIN num_days
      USING (bin) )
    GROUP BY flag, year, cat
  ),
  
  #
  # Maximum daily number in each half-month period in 2016
  # for North Korean lighting vessels (detected by light time optical imagery)
  #
  prk_lighting2016 AS (
    SELECT 
      bin,
      MAX(detection) AS half_month_max
    FROM data_leapyear
    WHERE year = 2016
      AND flag = "prk"
      AND sensor IN ("viirs")
    GROUP BY bin
  ),
  
  #
  # Estimated total fishing days in 2016
  # for North Korean lighting vessels (detected by light time optical imagery)
  #
  prk_lighting2016_fdays AS (
    SELECT
      'prk' AS flag, 2016 AS year, 'lighting' AS cat, 
      SUM(total) AS total, CAST(NULL AS BOOLEAN) AS sanction
    FROM (
      SELECT 
        bin, half_month_max * days AS total
      FROM prk_lighting2016
      LEFT JOIN num_days_leapyear
      USING (bin) )
    GROUP BY flag, year, cat
  ),
  
  #
  # Maximum daily number in each half-month period in 2017
  # for North Korean lighting vessels (detected by light time optical imagery)
  #
  prk_lighting2017 AS (
    SELECT 
      bin,
      MAX(detection) AS half_month_max
    FROM data
    WHERE year = 2017
      AND flag = "prk"
      AND sensor IN ("viirs")
    GROUP BY bin
  ),
  
  #
  # Estimated total fishing days in 2017
  # for North Korean lighting vessels (detected by light time optical imagery)
  #
  prk_lighting2017_fdays AS (
    SELECT
      'prk' AS flag, 2017 AS year, 'lighting' AS cat, 
      SUM(total) AS total, CAST(NULL AS BOOLEAN) AS sanction
    FROM (  
      SELECT bin, half_month_max * days AS total
      FROM prk_lighting2017
      LEFT JOIN num_days
      USING (bin) )
    GROUP BY flag, year, cat
  ),
  
  #
  # Maximum daily number in each half-month period in 2018
  # for North Korean lighting vessels (detected by light time optical imagery)
  #
  prk_lighting2018 AS (
    SELECT 
      bin,
      MAX(detection) AS half_month_max
    FROM data
    WHERE year = 2018
      AND flag = "prk"
      AND sensor IN ("viirs")
    GROUP BY bin
  ),
  
  #
  # Estimated total fishing days in 2018
  # for North Korean lighting vessels (detected by light time optical imagery)
  #
  prk_lighting2018_fdays AS (
    SELECT
      'prk' AS flag, 2018 AS year, 'lighting' AS cat, 
      SUM(total) AS total, CAST(NULL AS BOOLEAN) AS sanction
    FROM ( 
      SELECT bin, half_month_max * days AS total
      FROM prk_lighting2018
      LEFT JOIN num_days
      USING (bin) )
    GROUP BY flag, year, cat
  )
  

SELECT
  flag AS Flag_of_state_iso3,
  year AS Year,
  cat AS Vessel_class,
  total AS Estimated_total_days_of_fishing,
  sanction AS after_sanctions_imposed
FROM (
  SELECT *
  FROM chn_total2017_fdays
  UNION ALL
  SELECT *
  FROM chn_total2017_fdays_sanction
  UNION ALL
  SELECT *
  FROM chn_total2018_fdays
  UNION ALL
  SELECT *
  FROM prk_lighting2015_fdays
  UNION ALL
  SELECT *
  FROM prk_lighting2016_fdays
  UNION ALL
  SELECT *
  FROM prk_lighting2017_fdays
  UNION ALL
  SELECT *
  FROM prk_lighting2018_fdays )
ORDER BY flag, sanction, year, cat
"""
pd.read_gbq(q, project_id='global-fishing-watch', dialect='standard')

,Flag_of_state_iso3,Year,Vessel_class,Estimated_total_days_of_fishing,after_sanctions_imposed
0,chn,2017,lighting,8766.0,False
1,chn,2017,trawler,82618.0,False
2,chn,2017,lighting,5962.0,True
3,chn,2017,trawler,42934.0,True
4,chn,2018,lighting,6570.0,True
5,chn,2018,trawler,60689.0,True
6,prk,2015,lighting,38522.0,None
7,prk,2016,lighting,61552.0,None
8,prk,2017,lighting,141197.0,None
9,prk,2018,lighting,221993.0,None


## Chinese Vessel Counts by South Korean Coast Guard When Passing Through S.Korean Waters

In [4]:
#
# Import monthly observation data of S.Korean coast guard
#
q = """
SELECT *
FROM `paper_dark_fishing_fleets_in_north_korea.daily_number_of_detections` 
WHERE sensor = 'coastguard'
"""
df_cg = pd.read_gbq(q, project_id="global-fishing-watch", dialect='standard')

In [5]:
#
# Pull the coast guard's counts of Chinese vessels for each year
#
c2017 = list(df_cg[df_cg['year']==2017]['detection'])
c2018 = list(df_cg[df_cg['year']==2018]['detection'])
c2017_index = list(df_cg[df_cg['year']==2017]['day_of_year'])
c2018_index = list(df_cg[df_cg['year']==2018]['day_of_year'])

In [6]:
#
# Calculate fishing days by all Chinese fishing vessels in N.Korean waters
# based on the South Korean Coast Guard's monthly counts
#
import calendar
dnum = []
for y in range(2017,2019,1):
    d = []
    for m in range(1,13,1):
        d.append(calendar.monthrange(y,m)[1])
    dnum.append(d)
cguard = [c2017,c2018]
for y in range(2):
    prev=0
    ytotal = 0
    for n,vcount in enumerate(cguard[y]):
        ytotal += (vcount+prev)/2.0 * (dnum[y][n])
        prev = vcount
    print('Estimated total fishing days in {}: {}'.format(2017+y, ytotal))

Estimated total fishing days in 2017: 130463.5
Estimated total fishing days in 2018: 74316.5
